In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
pressures = np.load("../data/initial kernels/new_kernel/pressure.npy")
a_array = np.load("../data/initial kernels/new_kernel/pore_sizes.npy")
kernel = np.load("../data/initial kernels/new_kernel/kernel.npy")

In [5]:
for i in range(a_array.shape[0]):
    if a_array[i] < 3:
        if i % 5  == 0:
            plt.plot(pressures, kernel[i], label=f"{a_array[i]}", marker=".")
    if a_array[i] > 3:
        if i % 20  == 0:
            plt.plot(pressures, kernel[i], label=f"{a_array[i]}", marker=".")
plt.grid(True)
plt.legend()
plt.xscale("log")
plt.show()

In [135]:
def find_jump(x, y, threshold=0.1):
    from scipy.ndimage import gaussian_filter1d

    y_smooth = gaussian_filter1d(y, sigma=2)
    grad = np.gradient(y_smooth, x)
    grad2 = np.gradient(grad, x)


    jump_indices = np.where(np.abs(grad2) > threshold)[0]

    if len(jump_indices) == 0:
        return None
    else:
        return x[jump_indices[0]+1]

In [141]:
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import PchipInterpolator, UnivariateSpline

new_kernel = []
x = np.log(pressures)



for i in range(a_array.shape[0]):

    jump_x = find_jump(x, kernel[i], threshold=20000)
    if i == 100:
        print(np.exp(jump_x))

    smoothness = 0.001
    if jump_x is not None and sum(left_mask) != 0:
        left_mask = x < jump_x
        right_mask = x >= jump_x

        spline_left = UnivariateSpline(x[left_mask], kernel[i][left_mask], s=smoothness * sum(left_mask))
        spline_right = UnivariateSpline(x[right_mask], kernel[i][right_mask], s=smoothness * sum(right_mask))

        y_smooth = np.where(left_mask, spline_left(x), spline_right(x))
    else:
        spline = UnivariateSpline(x, kernel[i], s=smoothness * len(x))
        y_smooth = spline(x)


    new_kernel.append(y_smooth)

0.9513266715224342


In [142]:
plt.plot(pressures, kernel[10], marker=".")
plt.plot(pressures, new_kernel[10], marker=".")
plt.grid(True)
plt.legend()
plt.xscale("log")
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [8]:
for i in range(a_array.shape[0]):
    if a_array[i] < 3:
        if i % 5  == 0:
            plt.plot(pressures, new_kernel[i], label=f"{a_array[i]}", marker=".")
    if a_array[i] > 3:
        if i % 20  == 0:
            plt.plot(pressures, new_kernel[i], label=f"{a_array[i]}", marker=".")
plt.grid(True)
plt.legend()
plt.xscale("log")
plt.show()

In [4]:
pressures_cut_tabels = []
thresholds = [0.5, 0.8, 0.9, 0.95, 0.99]

for threshold in thresholds:
    pressures_cut_tabel = []
    for i in range(a_array.shape[0]):
        for j in range(pressures.shape[0]):
            if kernel[i][j] > max(kernel[i])*threshold:
                pressures_cut_tabel.append(pressures[j])
                break
    pressures_cut_tabels.append(pressures_cut_tabel)

In [5]:
for i in range(len(pressures_cut_tabels[1])):
    print(pressures_cut_tabel[i], a_array[i])

0.6948714490209711 0.7
0.6948714490209711 0.7125
0.48913569857225103 0.725
0.45090579423968047 0.7375
0.514682716615873 0.75
0.4636374608599723 0.7625
0.7598151874538369 0.775
0.3494773813917584 0.7875
0.3747645004879442 0.8
0.5402767836005385 0.8125
0.45090579423968047 0.825
0.5402767836005385 0.8375
0.6044757175322423 0.85
0.6689822522045161 0.8625
0.78588115512815 0.875
0.864386908961983 0.8875
0.9529114081200515 0.9
0.838166276070767 0.925
0.6560555360636577 0.95
0.6689822522045161 0.975
0.5659203443910349 1.0
0.45090579423968047 1.05
0.31163271255203173 1.1
0.4127826947333826 1.15
0.8250759790048957 1.2
0.838166276070767 1.3
0.22372398775989313 1.4
0.4636374608599723 1.5
0.9270675147657117 1.6
0.7337992703131253 1.7
0.8119976493404235 1.8
0.08665326219384657 2.0
0.5274734983599463 2.2
0.16126192898169248 2.4
0.838166276070767 2.6
0.6431426202020313 2.8
0.29904065055975326 3.0
0.7728409710632618 3.2
0.6689822522045161 3.4
0.7989331079151208 3.6
0.8250759790048957 3.8
0.798933107915

In [7]:
def plot_cut_tabels(pressures_cut_tabels):
    for i in range(len(pressures_cut_tabels)):
        #plt.plot(a_array, pressures_cut_tabels[i], marker=".", label=f"threshold = {thresholds[i]} (p/p0)")
        plt.plot(pressures_cut_tabels[i], a_array, marker=".", label=f"threshold = {thresholds[i]} (p/p0)")
        plt.xscale("log")
        plt.yscale("log")
        plt.grid(True)
        plt.xlabel("pore size (nm)")
        plt.ylabel("cut pressure (p/p0)")
        plt.legend()
        plt.show()

plot_cut_tabels(pressures_cut_tabels)

In [87]:
for k in range(len(pressures_cut_tabels)):
    for i in range(len(pressures_cut_tabel)-2, -1 , -1):
        if pressures_cut_tabels[k][i] > pressures_cut_tabels[k][i+1]:
            pressures_cut_tabels[k][i] = pressures_cut_tabels[k][i+1]

In [88]:
plot_cut_tabels(pressures_cut_tabels)

In [66]:
final_tabel = np.zeros((a_array.shape[0], 2))
for i in range(len(pressures_cut_tabels[1])):
    final_tabel[i] = np.array((pressures_cut_tabels[1][i], a_array[i]))
final_tabel

array([[4.98959096e-05, 7.00000000e-01],
       [6.27987245e-05, 7.12500000e-01],
       [6.27987245e-05, 7.25000000e-01],
       [7.90380443e-05, 7.37500000e-01],
       [9.94768869e-05, 7.50000000e-01],
       [1.57578628e-04, 7.62500000e-01],
       [1.98330945e-04, 7.75000000e-01],
       [2.49615673e-04, 7.87500000e-01],
       [3.14166790e-04, 8.00000000e-01],
       [4.97662767e-04, 8.12500000e-01],
       [7.88334656e-04, 8.25000000e-01],
       [1.24876539e-03, 8.37500000e-01],
       [1.57174882e-03, 8.50000000e-01],
       [2.48983767e-03, 8.62500000e-01],
       [2.48983767e-03, 8.75000000e-01],
       [2.48983767e-03, 8.87500000e-01],
       [2.48983767e-03, 9.00000000e-01],
       [2.48983767e-03, 9.25000000e-01],
       [2.48983767e-03, 9.50000000e-01],
       [2.48983767e-03, 9.75000000e-01],
       [2.48983767e-03, 1.00000000e+00],
       [2.48983767e-03, 1.05000000e+00],
       [2.48983767e-03, 1.10000000e+00],
       [6.24851412e-03, 1.15000000e+00],
       [1.568462

In [77]:
np.save(f"../data/initial kernels/new_kernel/cut_tabel", final_tabel)